In [5]:
import tweepy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import datetime
import string
import nltk
import warnings
import textblob
from textblob import TextBlob
import xlsxwriter
%matplotlib inline

warnings.filterwarnings('ignore')

consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

cursor = tweepy.Cursor(api.user_timeline, id="Akbank", tweet_mode="extended").items(10)
tweets=[]
time = []
likes = []
for i in cursor:
    tweets.append(i.full_text)
    time.append(i.created_at)
    likes.append(i.favorite_count)

print('Found %d tweets' % len(tweets))



Found 10 tweets


In [70]:
df = pd.DataFrame({'tweets':tweets,'time':time,'likes':likes})

In [7]:
df

,tweets,time,likes:
0,"İlk arkadaşlarımız, ilk sırdaşlarımız, ilk kah...",2021-05-08 21:03:41,18
1,Şubelerimizde 6-7-10 ve 11 Mayıs tarihlerinde ...,2021-05-06 14:08:23,12
2,Dünyanın en prestijli tasarım yarışmalarından ...,2021-05-03 12:07:35,9
3,Fırsatları duyan mobilden Akbanklı oluyor. Fey...,2021-05-01 08:02:59,35
4,30 Nisan 2021 tarihinden itibaren şubelerimizd...,2021-04-29 13:35:00,11
5,Kurumsal İletişim Bölüm Başkanımız Murat Göllü...,2021-04-28 12:33:00,3
6,Akbank’ın katkılarıyla düzenlenen MediaCat Liv...,2021-04-28 12:32:58,5
7,Pandemi nedeniyle sağlık önlemlerine dikkat ed...,2021-04-28 08:45:07,6
8,Etkinliği Marketing Türkiye’nin hesaplarından ...,2021-04-26 12:48:29,2
9,Marketing Türkiye ve Deloitte’un tasarladığı “...,2021-04-26 12:48:27,5


In [8]:
tweets

['İlk arkadaşlarımız, ilk sırdaşlarımız, ilk kahramanlarımız... Sonsuz sevgileriyle her zaman hayatımıza ışık tutan annelerimizin Anneler Günü kutlu olsun. https://t.co/lpbfKdNCdW',
 "Şubelerimizde 6-7-10 ve 11 Mayıs tarihlerinde 09.00 - 12.30 ve 13.30 - 16.30, 12 Mayıs'ta 09.00 – 12:30 saatlerinde hizmet vereceğiz. Bu saatler dışındaki işlemler için 7/24 Akbank Mobil, Akbank Internet, 444 25 25 no’lu Müşteri İletişim Merkezimiz ve ATM’lerimize bekliyoruz. https://t.co/9eVuC7WdAc",
 'Dünyanın en prestijli tasarım yarışmalarından iF Design Award’da 3 ödül birden kazandık. Bu başarıda katkısı olan tüm Akbanklılara teşekkür ederiz. https://t.co/9fw58ArYyw',
 'Fırsatları duyan mobilden Akbanklı oluyor. Feyyaz büyük haberi bir miktar orman havasıyla birlikte bu videoda alıyor.  https://t.co/ABuguK6lty https://t.co/IxyW5didCw',
 '30 Nisan 2021 tarihinden itibaren şubelerimizde 10.00-12.30 ve 13.30-16.00 saatleri arasında hizmet vereceğiz. Bu saatler dışındaki bankacılık işlemleri için 7/24 h

In [49]:
#buyuk-kucuk donusumu
df['tweets'] = df['tweets'].apply(lambda x: " ".join(x.lower() for x in x.split()))

#noktalama işaretleri
df['tweets'] = df['tweets'].str.replace('[^\w\s]','')

#sayılar
df['tweets'] = df['tweets'].str.replace('\d','')
"""
from nltk.corpus import stopwords
sw = stopwords.words('turkish')
df['tweets'] = df['tweets'].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
"""
def cleanTxt(text):
    text = re.sub(r'@[A-Za-z0-9]+','',text) #removed @mentions
    text = re.sub(r'#','',text) #removin #
    text = re.sub(r'RT[\S]+','',text) # removing RT
    text = re.sub(r'https?:\/\/\S+','',text) #remove hyperlink
    
    return text
df['tweets'] = df['tweets'].apply(cleanTxt)

In [50]:
df['tweets']

0    ilk arkadaşlarımız ilk sırdaşlarımız ilk kahra...
1    şubelerimizde mayıs tarihlerinde mayısta saatl...
2    dünyanın prestijli tasarım yarışmalarından if ...
3    fırsatları duyan mobilden akbanklı oluyor feyy...
4    nisan tarihinden itibaren şubelerimizde saatle...
5    kurumsal iletişim bölüm başkanımız murat göllü...
6    akbankın katkılarıyla düzenlenen mediacat live...
7    pandemi nedeniyle sağlık önlemlerine dikkat ed...
8    etkinliği marketing türkiyenin hesaplarından c...
9    marketing türkiye deloitteun tasarladığı marke...
Name: tweets, dtype: object

In [53]:
print(df['tweets'][2])
print(df['tweets'][4])

dünyanın prestijli tasarım yarışmalarından if design awardda ödül birden kazandık başarıda katkısı olan akbanklılara teşekkür ederiz tcofwaryyw
nisan tarihinden itibaren şubelerimizde saatleri arasında hizmet vereceğiz saatler dışındaki bankacılık işlemleri hizmet veren akbank mobil akbank internet nolu müşteri iletişim merkezimiz atmlerimize bekliyoruz tcoanrgmlmw


In [54]:
for i in df['tweets']:
    print(i)
    blob1=TextBlob(i)
    blob_eng=blob1.translate(to="en")
    print(blob_eng.sentiment)

ilk arkadaşlarımız ilk sırdaşlarımız ilk kahramanlarımız sonsuz sevgileriyle zaman hayatımıza ışık tutan annelerimizin anneler günü kutlu olsun tcolpbfkdncdw
Sentiment(polarity=0.4083333333333334, subjectivity=0.55)
şubelerimizde mayıs tarihlerinde mayısta saatlerinde hizmet vereceğiz saatler dışındaki işlemler akbank mobil akbank internet nolu müşteri iletişim merkezimiz atmlerimize bekliyoruz tcoevucwdac
Sentiment(polarity=-0.1, subjectivity=0.1)
dünyanın prestijli tasarım yarışmalarından if design awardda ödül birden kazandık başarıda katkısı olan akbanklılara teşekkür ederiz tcofwaryyw
Sentiment(polarity=0.4, subjectivity=0.25)
fırsatları duyan mobilden akbanklı oluyor feyyaz büyük haberi bir miktar orman havasıyla birlikte videoda alıyor tcoabuguklty tcoixywdidcw
Sentiment(polarity=0.0, subjectivity=0.1)
nisan tarihinden itibaren şubelerimizde saatleri arasında hizmet vereceğiz saatler dışındaki bankacılık işlemleri hizmet veren akbank mobil akbank internet nolu müşteri iletişim m

In [68]:
positive_tweets=[]
positive_result=[]

negative_tweets=[]
negative_result=[]

neutral=[]
neutral_result=[]

for i in df['tweets']:
    blob1=TextBlob(i)
    blob_eng=blob1.translate(to="en")
    
    if(blob_eng.polarity>0):
        positive_tweets.append(i)
        positive_result.append(blob_eng.sentiment)
    elif(blob_eng.polarity<0):
        negative_tweets.append(i)
        negative_result.append(blob_eng.sentiment)
    else:
        neutral.append(i)
        neutral_result.append(blob_eng.sentiment)
        


        

In [73]:
df_eng=pd.DataFrame({'Positive':positive_tweets,"Positive_Result":positive_result})
df_eng

,Positive,Positive_Result
0,ilk arkadaşlarımız ilk sırdaşlarımız ilk kahra...,"(0.4083333333333334, 0.55)"
1,dünyanın prestijli tasarım yarışmalarından if ...,"(0.4, 0.25)"
2,akbankın katkılarıyla düzenlenen mediacat live...,"(0.13636363636363635, 0.5)"
3,etkinliği marketing türkiyenin hesaplarından c...,"(0.13636363636363635, 0.5)"
4,marketing türkiye deloitteun tasarladığı marke...,"(0.05000000000000002, 0.5)"


In [74]:
df_eng=pd.DataFrame({'Negative':negative_tweets,"Negative_Result":negative_result})
df_eng

,Negative,Negative_Result
0,şubelerimizde mayıs tarihlerinde mayısta saatl...,"(-0.1, 0.1)"
1,nisan tarihinden itibaren şubelerimizde saatle...,"(-0.05, 0.07500000000000001)"
2,pandemi nedeniyle sağlık önlemlerine dikkat ed...,"(-0.006249999999999999, 0.24375000000000002)"


In [75]:
df_eng=pd.DataFrame({'Neutral':neutral,"Neutral_Result":neutral_result})
df_eng

,Neutral,Neutral_Result
0,fırsatları duyan mobilden akbanklı oluyor feyy...,"(0.0, 0.1)"
1,kurumsal iletişim bölüm başkanımız murat göllü...,"(0.0, 0.0)"


In [71]:
workbook = xlsxwriter.Workbook("AkbankSAPCopy2" + ".xlsx")
worksheet = workbook.add_worksheet()
row = 0
for i in range(10):
    worksheet.write_string(row, 0, str(df['time'][i]))
    worksheet.write(row, 1, df['tweets'][i])
    worksheet.write_string(row, 2, str(df['likes'][i]))
    row += 1

workbook.close()
print("Excel file ready")

Excel file ready
